In [1]:
%matplotlib inline
from pylab import *
import matplotlib.pyplot as plt
import os
import numpy as np

In [2]:
import pyAgrum as gum
import pyAgrum.lib.notebook as gnb

In [4]:
bn=gum.BayesNet('WaterSprinkler')
print(bn)

BN{nodes: 0, arcs: 0, domainSize: 1, dim: 0}


In [5]:
a, b, c, d, e = [ bn.add(name, 2) for name in "abcde" ]
bn

(gum::BayesNet<double>@0x2241142af90) BN{nodes: 5, arcs: 0, domainSize: 32, dim: 10}

In [6]:
for link in [(a,c),(b,c),(b,e),(c,d)]:
    bn.addArc(*link)
print(bn)

BN{nodes: 5, arcs: 4, domainSize: 32, dim: 20}


In [7]:
bn

(gum::BayesNet<double>@0x2241142af90) BN{nodes: 5, arcs: 4, domainSize: 32, dim: 20}

In [8]:
bn.cpt(a).fillWith([.6,.4])
bn.cpt(b).fillWith([.7,.3])

bn.cpt(e)[:]=[[.9,.1], [.1,.9]]

In [12]:
print(bn.cpt(c))

<c:0|a:0|b:0> :: 0 /<c:1|a:0|b:0> :: 0 /<c:0|a:1|b:0> :: 0 /<c:1|a:1|b:0> :: 0 /<c:0|a:0|b:1> :: 0 /<c:1|a:0|b:1> :: 0 /<c:0|a:1|b:1> :: 0 /<c:1|a:1|b:1> :: 0


In [13]:
#print(bn.cpt(c))
bn.cpt(c)[0,0,:] = [.8, .2] # b=0,a=0
bn.cpt(c)[0,1,:] = [.2, .8]
bn.cpt(c)[1,0,:] = [.2, .8]
bn.cpt(c)[1,1,:] = [.8, .2]
bn.cpt(c)

(gum::Potential<double>@0x22417b00990) <c:0|a:0|b:0> :: 0.8 /<c:1|a:0|b:0> :: 0.2 /<c:0|a:1|b:0> :: 0.2 /<c:1|a:1|b:0> :: 0.8 /<c:0|a:0|b:1> :: 0.2 /<c:1|a:0|b:1> :: 0.8 /<c:0|a:1|b:1> :: 0.8 /<c:1|a:1|b:1> :: 0.2

In [14]:
bn.cpt(d)[:]=[[.2,.8], [.8,.2]]

## Question 5

In [26]:
ie=gum.LazyPropagation(bn)
ie.makeInference()

In [27]:
ie.setEvidence({'c':1})

In [24]:
ie.posterior(a)*ie.posterior(d)*ie.posterior(e)*ie.posterior(b)

(gum::Potential<double>@0x22417b006b0) <b:0|e:0|d:0|a:0> :: 0.153145 /<b:1|e:0|d:0|a:0> :: 0.0835335 /<b:0|e:1|d:0|a:0> :: 0.0948039 /<b:1|e:1|d:0|a:0> :: 0.0517112 /<b:0|e:0|d:1|a:0> :: 0.0382862 /<b:1|e:0|d:1|a:0> :: 0.0208834 /<b:0|e:1|d:1|a:0> :: 0.023701 /<b:1|e:1|d:1|a:0> :: 0.0129278 /<b:0|e:0|d:0|a:1> :: 0.166578 /<b:1|e:0|d:0|a:1> :: 0.090861 /<b:0|e:1|d:0|a:1> :: 0.10312 /<b:1|e:1|d:0|a:1> :: 0.0562473 /<b:0|e:0|d:1|a:1> :: 0.0416446 /<b:1|e:0|d:1|a:1> :: 0.0227152 /<b:0|e:1|d:1|a:1> :: 0.02578 /<b:1|e:1|d:1|a:1> :: 0.0140618

In [37]:
pABCDE=bn.cpt("a")*bn.cpt("b")*bn.cpt("c")*bn.cpt("d")*bn.cpt("e")
pABCD=pABCDE.margSumOut(["c"])
pABCD

(gum::Potential<double>@0x22417b3dd60) <b:0|e:0|d:0|a:0> :: 0.12096 /<b:1|e:0|d:0|a:0> :: 0.01224 /<b:0|e:1|d:0|a:0> :: 0.01344 /<b:1|e:1|d:0|a:0> :: 0.11016 /<b:0|e:0|d:1|a:0> :: 0.25704 /<b:1|e:0|d:1|a:0> :: 0.00576 /<b:0|e:1|d:1|a:0> :: 0.02856 /<b:1|e:1|d:1|a:0> :: 0.05184 /<b:0|e:0|d:0|a:1> :: 0.17136 /<b:1|e:0|d:0|a:1> :: 0.00384 /<b:0|e:1|d:0|a:1> :: 0.01904 /<b:1|e:1|d:0|a:1> :: 0.03456 /<b:0|e:0|d:1|a:1> :: 0.08064 /<b:1|e:0|d:1|a:1> :: 0.00816 /<b:0|e:1|d:1|a:1> :: 0.00896 /<b:1|e:1|d:1|a:1> :: 0.07344

In [38]:
# La distribution de probabilité ABDE sachant C=1 est différente de la probabilité obtenus en marginalisant C

## Question 9

In [39]:
ie0=gum.LazyPropagation(bn)
ie0.makeInference()
ie0.posterior("e")

(gum::Potential<double>@0x22417b3d980) <e:0> :: 0.66 /<e:1> :: 0.34

In [40]:
ie2=gum.LazyPropagation(bn)
ie2.makeInference()
ie2.setEvidence({"a":0,"d":0})
ie2.posterior("e")

(gum::Potential<double>@0x22417b3de80) <e:0> :: 0.518692 /<e:1> :: 0.481308

In [41]:
ie3=gum.LazyPropagation(bn)
ie3.makeInference()
ie3.setEvidence({"d":1})
ie3.posterior("a")*ie3.posterior("b")

(gum::Potential<double>@0x22417b00a10) <b:0|a:0> :: 0.48664 /<b:1|a:0> :: 0.180545 /<b:0|a:1> :: 0.242753 /<b:1|a:1> :: 0.0900619